# DENOISING

La reducción de ruido es un proceso que consiste en eliminar el ruido de una señal. Existen técnicas de reducción de ruido para audio e imágenes, cuyos algoritmos tienden a alterar las señales en mayor o menor grado.

## Carga de librerias

In [1]:
import os, glob, random, time, sys, pickle, gc
from scipy.stats import wilcoxon
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa, librosa.display
import soundfile as sf
from scipy.optimize import minimize
import ast
from metrics import AudioMetrics
from metrics import AudioMetrics2
from scipy import signal
import wavfile
from nnAudio.Spectrogram import CQT1992v2
import noise_addition_utils
import colorednoise as cn
import cv2
import audioread
import logging
from contextlib import contextmanager
from joblib import Parallel, delayed
from pathlib import Path
from typing import Optional
import IPython
import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, WeightedRandomSampler
import torch.utils.data as torchdata
# import torch.optim as optim
import torch_optimizer as optim
from torchvision import transforms, models, datasets
# from torchlibrosa.stft import Spectrogram, LogmelFilterBank
# from torchlibrosa.augmentation import SpecAugmentation
from torchaudio.transforms import AmplitudeToDB
from transformers import get_linear_schedule_with_warmup
import transformers
from torch.cuda.amp import autocast, GradScaler

from albumentations.core.transforms_interface import ImageOnlyTransform
from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation
import pywt

from tqdm import tqdm
# from tqdm.notebook import tqdm
from functools import partial
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, GroupKFold, train_test_split
from sklearn.metrics import multilabel_confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay, confusion_matrix, mean_squared_error, precision_recall_fscore_support
from skimage.transform import resize

import albumentations as A
import albumentations.pytorch.transforms as T
from IPython.display import clear_output

from PIL import Image, ImageOps
import timm
from timm.models.efficientnet import tf_efficientnet_b0_ns, tf_efficientnet_b1_ns, tf_efficientnet_b2_ns, tf_efficientnet_b3_ns, tf_efficientnet_b4_ns
from pathlib import Path
import os

import warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.__version__, np.__version__, pd.__version__

C:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
C:\Users\Ramón Sieira\AppData\Roaming\Python\Python39\site-packages\nnAudio\Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


('1.12.1', '1.21.5', '1.4.4')

In [2]:
tramo_length = 2.0
  
device = ('cuda' if torch.cuda.is_available() else 'cpu')

## Creacion de la BD

In [3]:
if True:
    dataset1 = pd.read_csv("https://storage.googleapis.com/datathon2022/labels_dataset1_v2.csv")
    print(dataset1.shape)
    df = dataset1.query('label!="volcano"').drop_duplicates()
    print(df.shape)
    df.label.value_counts()

    len_audio = []
    sr_audio = []
    seconds_audio = []
    for name_file in tqdm(df.path.unique()):
        y, sr = sf.read(f'../input/datathon2022/dataset1/{name_file}.wav')
        len_audio.append(len(y))
        sr_audio.append(sr)
        seconds_audio.append(len(y)/sr)
    df['len_audio'] = df['path'].map(dict(zip(df.path.unique(), len_audio)))
    df['sr_audio'] = df['path'].map(dict(zip(df.path.unique(), sr_audio)))
    df['seconds_audio'] = df['path'].map(dict(zip(df.path.unique(), seconds_audio)))
    df['dir'] = ['../input/datathon2022/dataset1/'+i+'.wav' for i in df['path']]
    df['clase'] = df['label'].values
    clases = dict(df['clase'].value_counts())
    clases_num = dict(zip(list(clases.keys()), np.arange(len(clases))))
    df['clase_num'] = df['clase'].map(clases_num)
    labels = pd.get_dummies(df['clase'])
    
    db_troceada = []
    labels_troceada = []
    step_audio = int(tramo_length * 50000)
    for namefile in tqdm(df['path'].unique()):
        df_file = df.query(f'path=="{namefile}"').reset_index()
        df_file['start_pos'] = df_file[['start']].values * 50000
        df_file['end_pos'] = df_file[['end']].values * 50000
        df_file['start_pos']  = df_file['start_pos'].astype(int)
        df_file['end_pos']  = df_file['end_pos'].astype(int)
        for pos_ini in np.arange(0,14700000,step_audio):
            pos_fin = pos_ini + step_audio
            if pos_fin > 14700000:
                pos_fin = 14700000
            pos_dentro = (pos_ini>df_file['start_pos'].values) & (pos_ini<df_file['end_pos'].values)
            labels_list = []
            if np.sum(pos_dentro>0):
                labels_list = list(np.unique(df_file.loc[pos_dentro, 'label'].values))
#                 print(namefile, pos_ini, pos_fin, labels)
            db_troceada.append(dict(namefile=namefile,
                                    dir=f'../input/datathon2022/dataset1/{namefile}.wav',
                                    start=float(pos_ini/50000),
                                    end=float(pos_fin/50000),
                                    pos_ini=pos_ini,
                                    pos_fin=pos_fin,
                                    labels=labels_list))
            labels_row = np.zeros(len(clases_num))
            for label in labels_list:
                labels_row[clases_num[label]] = 1.0
            labels_troceada.append(labels_row)
            
    db_troceada = pd.DataFrame(db_troceada)
    labels_troceada = pd.DataFrame(np.stack(labels_troceada))
    labels_troceada.columns = list(clases_num.keys())

    with open('../input/database_datathon2022.pickle', 'wb') as f:
        pickle.dump(df, f)
        pickle.dump(clases, f)
        pickle.dump(clases_num, f)
        pickle.dump(labels, f)
        pickle.dump(db_troceada, f)
        pickle.dump(labels_troceada, f)
else:
    with open('../input/database_datathon2022.pickle', 'rb') as f:
        df = pickle.load(f)
        clases = pickle.load(f)
        clases_num = pickle.load(f)
        labels = pickle.load(f)
        db_troceada = pickle.load(f)
        labels_troceada = pickle.load(f)
df.head()

(5459, 5)
(5423, 5)


100%|███████████████████████████████████████████████████████████████████████████████| 452/452 [00:03<00:00, 128.01it/s]


,path,start,duration,end,label,len_audio,sr_audio,seconds_audio,dir,clase,clase_num
0,012213d40662f86e8f775379bbf94dec,61.186,2.500,63.686,whistle,14700000,50000,294.0,../input/datathon2022/dataset1/012213d40662f86...,whistle,0
1,0159af9f3ca04ada46f0c353ac210bb8,76.260,17.000,93.260,cetaceans_allfreq,14700000,50000,294.0,../input/datathon2022/dataset1/0159af9f3ca04ad...,cetaceans_allfreq,2
2,0159af9f3ca04ada46f0c353ac210bb8,161.482,1.000,162.482,click,14700000,50000,294.0,../input/datathon2022/dataset1/0159af9f3ca04ad...,click,1
3,0271a0818427d7fae7afde67bf49ba30,91.563,1.000,92.563,click,14700000,50000,294.0,../input/datathon2022/dataset1/0271a0818427d7f...,click,1
4,0271a0818427d7fae7afde67bf49ba30,125.627,1.316,126.943,click,14700000,50000,294.0,../input/datathon2022/dataset1/0271a0818427d7f...,click,1


In [4]:
df_train_orig = db_troceada.reset_index(drop=True)
labels_train_orig = labels_troceada
df_train_orig.shape, labels_train_orig.shape

((66444, 7), (66444, 4))

Aqui se ven los datos separados donde hay o no evento es decir ruido o evento

In [5]:
hay_evento = (labels_train_orig.sum(axis=1)>0)
hay_evento.value_counts()

False    58611
True      7833
dtype: int64

Sample del dataframe, como vemos puede haber labels a la vez para un mismo intervalo, en este caso hemos particionado los eventos/no eventos en archivos de 2s

In [6]:
df_train_orig.loc[hay_evento].sample(3)

,namefile,dir,start,end,pos_ini,pos_fin,labels
21635,5849e10d3febc43b576290f2f6706ddd,../input/datathon2022/dataset1/5849e10d3febc43...,52.0,54.0,2600000,2700000,[cetaceans_allfreq]
1672,083edafa2b1d1f1944018ca7d2e28e97,../input/datathon2022/dataset1/083edafa2b1d1f1...,110.0,112.0,5500000,5600000,[allfreq]
57920,ddc7eeb968642eb5fc9f28fe3fd9e718,../input/datathon2022/dataset1/ddc7eeb968642eb...,4.0,6.0,200000,300000,[whistle]


# MODELO DEEP LEARNING

## Creacion del Dataset de entramiento train/test

Para realizar el entrenamiento generamos 1000 archivos por cada seccion de entrada/train-salida/train-test

In [7]:
unicos_names = df_train_orig.namefile.unique()
print(len(unicos_names))
primeros = unicos_names[:100]
segundos = unicos_names[100:200]
terceros = unicos_names[200:300]
noeventos = df_train_orig.loc[~hay_evento]

df1 = noeventos[noeventos.namefile.isin(primeros)].sample(100)
df2 = noeventos[noeventos.namefile.isin(segundos)].sample(100)
df3 = noeventos[noeventos.namefile.isin(terceros)].sample(100)

452


In [8]:
df1.shape, df2.shape, df3.shape

((100, 7), (100, 7), (100, 7))

## Almacenamiento de los recortes

In [9]:
for nrow, record in tqdm(df1.loc[~hay_evento].iterrows()):
    path = record['dir']
    namefile = record['namefile']+'_'+str(record['pos_ini'])
    y, sr = sf.read(path)
    y = y[record['pos_ini']:record['pos_fin']]
    sf.write('../Noise2Noise-audio_denoising_without_clean_training_data-main/Datasets/noise_marine_train_salida/'+namefile+'.wav',
             y, sr)

100it [00:04, 24.50it/s]


In [10]:
for nrow, record in tqdm(df2.loc[~hay_evento].iterrows()):
    path = record['dir']
    namefile = record['namefile']+'_'+str(record['pos_ini'])
    y, sr = sf.read(path)
    y = y[record['pos_ini']:record['pos_fin']]
    sf.write('../Noise2Noise-audio_denoising_without_clean_training_data-main/Datasets/noise_marine_train_entrada/'+namefile+'.wav',
             y, sr)

100it [00:04, 24.48it/s]


In [11]:
for nrow, record in tqdm(df3.loc[~hay_evento].iterrows()):
    path = record['dir']
    namefile = record['namefile']+'_'+str(record['pos_ini'])
    y, sr = sf.read(path)
    y = y[record['pos_ini']:record['pos_fin']]
    sf.write('../Noise2Noise-audio_denoising_without_clean_training_data-main/Datasets/noise_marine_test/'+namefile+'.wav',
             y, sr)

100it [00:04, 24.35it/s]


## Creacion/Definicion del modelo + Carga de los datos

In [12]:
noise_class = "marine"

In [13]:
training_type =  "Noise2Noise" 

In [14]:
TRAIN_INPUT_DIR = Path('Datasets/noise_marine_train_entrada')
TRAIN_TARGET_DIR = Path('Datasets/noise_marine_train_salida/')
TEST_NOISY_DIR = Path('Datasets/noise_marine_test/')
TEST_CLEAN_DIR = Path('Datasets/noise_marine_test/')

In [15]:
basepath = str(noise_class)+"_"+training_type
os.makedirs(basepath,exist_ok=True)
os.makedirs(basepath+"/Weights",exist_ok=True)
os.makedirs(basepath+"/Samples",exist_ok=True)

In [16]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')
       
DEVICE = torch.device('cuda' if train_on_gpu else 'cpu')

Training on GPU.


In [17]:
SAMPLE_RATE = 48000
N_FFT = (SAMPLE_RATE * 64) // 1000 
HOP_LENGTH = (SAMPLE_RATE * 16) // 1000 

In [18]:
class SpeechDataset(Dataset):
    def __init__(self, noisy_files, clean_files, n_fft=64, hop_length=16):
        super().__init__()
        # list of files
        self.noisy_files = sorted(noisy_files)
        self.clean_files = sorted(clean_files)
        
        # stft parameters
        self.n_fft = n_fft
        self.hop_length = hop_length
        
        self.len_ = len(self.noisy_files)
        
        # fixed len
        self.max_len = 165000

    
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        waveform, _ = torchaudio.load(file)
        return waveform
    
    def __getitem__(self, index):
        # load to tensors and normalization
        x_clean = self.load_sample(self.clean_files[index])
        x_noisy = self.load_sample(self.noisy_files[index])
        
        # padding/cutting
        x_clean = self._prepare_sample(x_clean)
        x_noisy = self._prepare_sample(x_noisy)
        
        # Short-time Fourier transform
        x_noisy_stft = torch.stft(input=x_noisy, n_fft=self.n_fft, 
                                  hop_length=self.hop_length, normalized=True)
        x_clean_stft = torch.stft(input=x_clean, n_fft=self.n_fft, 
                                  hop_length=self.hop_length, normalized=True)
        
        return x_noisy_stft, x_clean_stft
        
    def _prepare_sample(self, waveform):
        waveform = waveform.numpy()
        current_len = waveform.shape[1]
        
        output = np.zeros((1, self.max_len), dtype='float32')
        output[0, -current_len:] = waveform[0, :self.max_len]
        output = torch.from_numpy(output)
        
        return output

In [19]:
train_input_files = sorted(list(TRAIN_INPUT_DIR.rglob('*.wav')))[:200]
train_target_files = sorted(list(TRAIN_TARGET_DIR.rglob('*.wav')))[:200]

test_noisy_files = sorted(list(TEST_NOISY_DIR.rglob('*.wav')))[:200]
test_clean_files = sorted(list(TEST_CLEAN_DIR.rglob('*.wav')))[:200]

print("No. of Training files:",len(train_input_files))
print("No. of Testing files:",len(test_noisy_files))

No. of Training files: 100
No. of Testing files: 100


In [20]:
test_dataset = SpeechDataset(test_noisy_files, test_clean_files, N_FFT, HOP_LENGTH)
train_dataset = SpeechDataset(train_input_files, train_target_files, N_FFT, HOP_LENGTH)

In [21]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader_single_unshuffled = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [22]:
def test_set_metrics(test_loader, model):
    metric_names = ["CSIG","CBAK","COVL","PESQ","SSNR","STOI"]
    overall_metrics = [[] for i in range(len(metric_names))]
    
    for i,(noisy,clean) in enumerate(test_loader):
        x_est = model(noisy.to(DEVICE), is_istft=True)
        x_est_np = x_est[0].view(-1).detach().cpu().numpy()
        x_c_np = torch.istft(torch.squeeze(clean[0], 1), n_fft=N_FFT, hop_length=HOP_LENGTH, normalized=True).view(-1).detach().cpu().numpy()
        metrics = AudioMetrics(x_c_np, x_est_np, SAMPLE_RATE)
        
        overall_metrics[0].append(metrics.CSIG)
        overall_metrics[1].append(metrics.CBAK)
        overall_metrics[2].append(metrics.COVL)
        overall_metrics[3].append(metrics.PESQ)
        overall_metrics[4].append(metrics.SSNR)
        overall_metrics[5].append(metrics.STOI)
    
    metrics_dict = dict()
    for i in range(len(metric_names)):
        metrics_dict[metric_names[i]] ={'mean': np.mean(overall_metrics[i]), 'std_dev': np.std(overall_metrics[i])} 
    
    return metrics_dict

In [23]:
class CConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        
        self.real_conv = nn.Conv2d(in_channels=self.in_channels, 
                                   out_channels=self.out_channels, 
                                   kernel_size=self.kernel_size, 
                                   padding=self.padding, 
                                   stride=self.stride)
        
        self.im_conv = nn.Conv2d(in_channels=self.in_channels, 
                                 out_channels=self.out_channels, 
                                 kernel_size=self.kernel_size, 
                                 padding=self.padding, 
                                 stride=self.stride)
        
        # Glorot initialization.
        nn.init.xavier_uniform_(self.real_conv.weight)
        nn.init.xavier_uniform_(self.im_conv.weight)
        
        
    def forward(self, x):
        x_real = x[..., 0]
        x_im = x[..., 1]
        
        c_real = self.real_conv(x_real) - self.im_conv(x_im)
        c_im = self.im_conv(x_real) + self.real_conv(x_im)
        
        output = torch.stack([c_real, c_im], dim=-1)
        return output

In [24]:
class CConvTranspose2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, output_padding=0, padding=0):
        super().__init__()
        
        self.in_channels = in_channels

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.output_padding = output_padding
        self.padding = padding
        self.stride = stride
        
        self.real_convt = nn.ConvTranspose2d(in_channels=self.in_channels, 
                                            out_channels=self.out_channels, 
                                            kernel_size=self.kernel_size, 
                                            output_padding=self.output_padding,
                                            padding=self.padding,
                                            stride=self.stride)
        
        self.im_convt = nn.ConvTranspose2d(in_channels=self.in_channels, 
                                            out_channels=self.out_channels, 
                                            kernel_size=self.kernel_size, 
                                            output_padding=self.output_padding, 
                                            padding=self.padding,
                                            stride=self.stride)
        
        
        # Glorot initialization.
        nn.init.xavier_uniform_(self.real_convt.weight)
        nn.init.xavier_uniform_(self.im_convt.weight)
        
        
    def forward(self, x):
        x_real = x[..., 0]
        x_im = x[..., 1]
        
        ct_real = self.real_convt(x_real) - self.im_convt(x_im)
        ct_im = self.im_convt(x_real) + self.real_convt(x_im)
        
        output = torch.stack([ct_real, ct_im], dim=-1)
        return output

In [25]:
class CBatchNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True):
        super().__init__()
        
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.track_running_stats = track_running_stats
        
        self.real_b = nn.BatchNorm2d(num_features=self.num_features, eps=self.eps, momentum=self.momentum,
                                      affine=self.affine, track_running_stats=self.track_running_stats)
        self.im_b = nn.BatchNorm2d(num_features=self.num_features, eps=self.eps, momentum=self.momentum,
                                    affine=self.affine, track_running_stats=self.track_running_stats) 
        
    def forward(self, x):
        x_real = x[..., 0]
        x_im = x[..., 1]
        
        n_real = self.real_b(x_real)
        n_im = self.im_b(x_im)  
        
        output = torch.stack([n_real, n_im], dim=-1)
        return output

In [26]:
class Encoder(nn.Module):
    def __init__(self, filter_size=(7,5), stride_size=(2,2), in_channels=1, out_channels=45, padding=(0,0)):
        super().__init__()
        
        self.filter_size = filter_size
        self.stride_size = stride_size
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.padding = padding

        self.cconv = CConv2d(in_channels=self.in_channels, out_channels=self.out_channels, 
                             kernel_size=self.filter_size, stride=self.stride_size, padding=self.padding)
        
        self.cbn = CBatchNorm2d(num_features=self.out_channels) 
        
        self.leaky_relu = nn.LeakyReLU()
            
    def forward(self, x):
        
        conved = self.cconv(x)
        normed = self.cbn(conved)
        acted = self.leaky_relu(normed)
        
        return acted

In [27]:
class Decoder(nn.Module):
    def __init__(self, filter_size=(7,5), stride_size=(2,2), in_channels=1, out_channels=45,
                 output_padding=(0,0), padding=(0,0), last_layer=False):
        super().__init__()
        
        self.filter_size = filter_size
        self.stride_size = stride_size
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.output_padding = output_padding
        self.padding = padding
        
        self.last_layer = last_layer
        
        self.cconvt = CConvTranspose2d(in_channels=self.in_channels, out_channels=self.out_channels, 
                             kernel_size=self.filter_size, stride=self.stride_size, output_padding=self.output_padding, padding=self.padding)
        
        self.cbn = CBatchNorm2d(num_features=self.out_channels) 
        
        self.leaky_relu = nn.LeakyReLU()
            
    def forward(self, x):
        
        conved = self.cconvt(x)
        
        if not self.last_layer:
            normed = self.cbn(conved)
            output = self.leaky_relu(normed)
        else:
            m_phase = conved / (torch.abs(conved) + 1e-8)
            m_mag = torch.tanh(torch.abs(conved))
            output = m_phase * m_mag
            
        return output

In [28]:
from pesq import pesq
from scipy import interpolate
def resample(original, old_rate, new_rate):
    if old_rate != new_rate:
        duration = original.shape[0] / old_rate
        time_old  = np.linspace(0, duration, original.shape[0])
        time_new  = np.linspace(0, duration, int(original.shape[0] * new_rate / old_rate))
        interpolator = interpolate.interp1d(time_old, original.T)
        new_audio = interpolator(time_new).T
        return new_audio
    else:
        return original

def wsdr_fn(x_, y_pred_, y_true_, eps=1e-8):
    # to time-domain waveform
    y_true_ = torch.squeeze(y_true_, 1)
    y_true = torch.istft(y_true_, n_fft=N_FFT, hop_length=HOP_LENGTH, normalized=True)
    x_ = torch.squeeze(x_, 1)
    x = torch.istft(x_, n_fft=N_FFT, hop_length=HOP_LENGTH, normalized=True)

    y_pred = y_pred_.flatten(1)
    y_true = y_true.flatten(1)
    x = x.flatten(1)


    def sdr_fn(true, pred, eps=1e-8):
        num = torch.sum(true * pred, dim=1)
        den = torch.norm(true, p=2, dim=1) * torch.norm(pred, p=2, dim=1)
        return -(num / (den + eps))

    # true and estimated noise
    z_true = x - y_true
    z_pred = x - y_pred

    a = torch.sum(y_true**2, dim=1) / (torch.sum(y_true**2, dim=1) + torch.sum(z_true**2, dim=1) + eps)
    wSDR = a * sdr_fn(y_true, y_pred) + (1 - a) * sdr_fn(z_true, z_pred)
    return torch.mean(wSDR)

wonky_samples = []

def getMetricsonLoader(loader, net, use_net=True):
    net.eval()
    # Original test metrics
    scale_factor = 32768
    # metric_names = ["CSIG","CBAK","COVL","PESQ","SSNR","STOI","SNR "]
    metric_names = ["PESQ-WB","PESQ-NB","SNR","SSNR","STOI"]
    overall_metrics = [[] for i in range(5)]
    for i, data in enumerate(loader):
        if (i+1)%10==0:
            end_str = "\n"
        else:
            end_str = ","
        #print(i,end=end_str)
        if i in wonky_samples:
            print("Something's up with this sample. Passing...")
        else:
            noisy = data[0]
            clean = data[1]
            if use_net: # Forward of net returns the istft version
                x_est = net(noisy.to(DEVICE), is_istft=True)
                x_est_np = x_est.view(-1).detach().cpu().numpy()
            else:
                x_est_np = torch.istft(torch.squeeze(noisy, 1), n_fft=N_FFT, hop_length=HOP_LENGTH, normalized=True).view(-1).detach().cpu().numpy()
            x_clean_np = torch.istft(torch.squeeze(clean, 1), n_fft=N_FFT, hop_length=HOP_LENGTH, normalized=True).view(-1).detach().cpu().numpy()
            
        
            metrics = AudioMetrics2(x_clean_np, x_est_np, 48000)
            
            ref_wb = resample(x_clean_np, 48000, 16000)
            deg_wb = resample(x_est_np, 48000, 16000)
            pesq_wb = pesq(16000, ref_wb, deg_wb, 'wb')
            
            ref_nb = resample(x_clean_np, 48000, 8000)
            deg_nb = resample(x_est_np, 48000, 8000)
            pesq_nb = pesq(8000, ref_nb, deg_nb, 'nb')

            #print(new_scores)
            #print(metrics.PESQ, metrics.STOI)

            overall_metrics[0].append(pesq_wb)
            overall_metrics[1].append(pesq_nb)
            overall_metrics[2].append(metrics.SNR)
            overall_metrics[3].append(metrics.SSNR)
            overall_metrics[4].append(metrics.STOI)
    print()
    print("Sample metrics computed")
    results = {}
    for i in range(5):
        temp = {}
        temp["Mean"] =  np.mean(overall_metrics[i])
        temp["STD"]  =  np.std(overall_metrics[i])
        temp["Min"]  =  min(overall_metrics[i])
        temp["Max"]  =  max(overall_metrics[i])
        results[metric_names[i]] = temp
    print("Averages computed")
    if use_net:
        addon = "(cleaned by model)"
    else:
        addon = "(pre denoising)"
    print("Metrics on test data",addon)
    for i in range(5):
        print("{} : {:.3f}+/-{:.3f}".format(metric_names[i], np.mean(overall_metrics[i]), np.std(overall_metrics[i])))
    return results

In [29]:
def train_epoch(net, train_loader, loss_fn, optimizer):
    net.train()
    train_ep_loss = 0.
    counter = 0
    for noisy_x, clean_x in train_loader:

        noisy_x, clean_x = noisy_x.to(DEVICE), clean_x.to(DEVICE)

        # zero  gradients
        net.zero_grad()

        # get the output from the model
        pred_x = net(noisy_x)

        # calculate loss
        loss = loss_fn(noisy_x, pred_x, clean_x)
        loss.backward()
        optimizer.step()

        train_ep_loss += loss.item() 
        counter += 1

    train_ep_loss /= counter

    # clear cache
    gc.collect()
    torch.cuda.empty_cache()
    return train_ep_loss

In [30]:
def test_epoch(net, test_loader, loss_fn, use_net=True):
    net.eval()
    test_ep_loss = 0.
    counter = 0.
    
    #print("Actual compute done...testing now")
    
    testmet = getMetricsonLoader(test_loader,net,use_net)

    # clear cache
    gc.collect()
    torch.cuda.empty_cache()
    
    return test_ep_loss, testmet

In [31]:
def train(net, train_loader, test_loader, loss_fn, optimizer, scheduler, epochs):
    
    train_losses = []
    test_losses = []

    for e in tqdm(range(epochs)):

        # first evaluating for comparison
        
        if e == 0 and training_type=="Noise2Clean":
            print("Pre-training evaluation")
            testmet = getMetricsonLoader(test_loader,net,False)    # again, modified cuz im loading
        
            with open(basepath + "/results.txt","w+") as f:
                f.write("Initial : \n")
                f.write(str(testmet))
                f.write("\n")
        
        
        train_loss = train_epoch(net, train_loader, loss_fn, optimizer)
        test_loss = 0
        scheduler.step()
        print("Saving model....")
        
        with torch.no_grad():
            test_loss, testmet = test_epoch(net, test_loader, loss_fn,use_net=True)

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
        #print("skipping testing cuz peak autism idk")
        
        with open(basepath + "/results.txt","a") as f:
            f.write("Epoch :"+str(e+1) + "\n" + str(testmet))
            f.write("\n")
        
        print("OPed to txt")
        
        torch.save(net.state_dict(), basepath +'/Weights/dc20_model_'+str(e+1)+'.pth')
        torch.save(optimizer.state_dict(), basepath+'/Weights/dc20_opt_'+str(e+1)+'.pth')
        
        print("Models saved")

        # clear cache
        torch.cuda.empty_cache()
        gc.collect()
    return train_loss, test_loss

In [32]:
class DCUnet20(nn.Module):
    """
    Deep Complex U-Net class of the model.
    """
    def __init__(self, n_fft=64, hop_length=16):
        super().__init__()
        
        # for istft
        self.n_fft = n_fft
        self.hop_length = hop_length
        
        self.set_size(model_complexity=int(45//1.414), input_channels=1, model_depth=20)
        self.encoders = []
        self.model_length = 20 // 2
        
        for i in range(self.model_length):
            module = Encoder(in_channels=self.enc_channels[i], out_channels=self.enc_channels[i + 1],
                             filter_size=self.enc_kernel_sizes[i], stride_size=self.enc_strides[i], padding=self.enc_paddings[i])
            self.add_module("encoder{}".format(i), module)
            self.encoders.append(module)

        self.decoders = []

        for i in range(self.model_length):
            if i != self.model_length - 1:
                module = Decoder(in_channels=self.dec_channels[i] + self.enc_channels[self.model_length - i], out_channels=self.dec_channels[i + 1], 
                                 filter_size=self.dec_kernel_sizes[i], stride_size=self.dec_strides[i], padding=self.dec_paddings[i],
                                 output_padding=self.dec_output_padding[i])
            else:
                module = Decoder(in_channels=self.dec_channels[i] + self.enc_channels[self.model_length - i], out_channels=self.dec_channels[i + 1], 
                                 filter_size=self.dec_kernel_sizes[i], stride_size=self.dec_strides[i], padding=self.dec_paddings[i],
                                 output_padding=self.dec_output_padding[i], last_layer=True)
            self.add_module("decoder{}".format(i), module)
            self.decoders.append(module)
       
        
    def forward(self, x, is_istft=True):
        # print('x : ', x.shape)
        orig_x = x
        xs = []
        for i, encoder in enumerate(self.encoders):
            xs.append(x)
            x = encoder(x)
            # print('Encoder : ', x.shape)
            
        p = x
        for i, decoder in enumerate(self.decoders):
            p = decoder(p)
            if i == self.model_length - 1:
                break
            # print('Decoder : ', p.shape)
            p = torch.cat([p, xs[self.model_length - 1 - i]], dim=1)
        
        # u9 - the mask
        
        mask = p
        
        # print('mask : ', mask.shape)
        
        output = mask * orig_x
        output = torch.squeeze(output, 1)


        if is_istft:
            output = torch.istft(output, n_fft=self.n_fft, hop_length=self.hop_length, normalized=True)
        
        return output

    
    def set_size(self, model_complexity, model_depth=20, input_channels=1):

        if model_depth == 20:
            self.enc_channels = [input_channels,
                                 model_complexity,
                                 model_complexity,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 128]

            self.enc_kernel_sizes = [(7, 1),
                                     (1, 7),
                                     (6, 4),
                                     (7, 5),
                                     (5, 3),
                                     (5, 3),
                                     (5, 3),
                                     (5, 3),
                                     (5, 3),
                                     (5, 3)]

            self.enc_strides = [(1, 1),
                                (1, 1),
                                (2, 2),
                                (2, 1),
                                (2, 2),
                                (2, 1),
                                (2, 2),
                                (2, 1),
                                (2, 2),
                                (2, 1)]

            self.enc_paddings = [(3, 0),
                                 (0, 3),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0)]

            self.dec_channels = [0,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity * 2,
                                 model_complexity,
                                 model_complexity,
                                 1]

            self.dec_kernel_sizes = [(6, 3), 
                                     (6, 3),
                                     (6, 3),
                                     (6, 4),
                                     (6, 3),
                                     (6, 4),
                                     (8, 5),
                                     (7, 5),
                                     (1, 7),
                                     (7, 1)]

            self.dec_strides = [(2, 1), #
                                (2, 2), #
                                (2, 1), #
                                (2, 2), #
                                (2, 1), #
                                (2, 2), #
                                (2, 1), #
                                (2, 2), #
                                (1, 1),
                                (1, 1)]

            self.dec_paddings = [(0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 0),
                                 (0, 3),
                                 (3, 0)]
            
            self.dec_output_padding = [(0,0),
                                       (0,0),
                                       (0,0),
                                       (0,0),
                                       (0,0),
                                       (0,0),
                                       (0,0),
                                       (0,0),
                                       (0,0),
                                       (0,0)]
        else:
            raise ValueError("Unknown model depth : {}".format(model_depth))

In [33]:
# # clear cache
gc.collect()
torch.cuda.empty_cache()

dcunet20 = DCUnet20(N_FFT, HOP_LENGTH).to(DEVICE)
optimizer = torch.optim.Adam(dcunet20.parameters())
loss_fn = wsdr_fn
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

## Entrenamiento del modelo

In [34]:
train_losses, test_losses = train(dcunet20, train_loader, test_loader, loss_fn, optimizer, scheduler, 4)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Saving model....


 25%|████████████████████▊                                                              | 1/4 [03:08<09:25, 188.60s/it]


Sample metrics computed
Averages computed
Metrics on test data (cleaned by model)
PESQ-WB : 3.462+/-0.845
PESQ-NB : 4.199+/-0.352
SNR : -3.571+/-0.740
SSNR : -5.958+/-0.420
STOI : 0.632+/-0.042
OPed to txt
Models saved
Saving model....


 50%|█████████████████████████████████████████▌                                         | 2/4 [06:09<06:08, 184.33s/it]


Sample metrics computed
Averages computed
Metrics on test data (cleaned by model)
PESQ-WB : 2.404+/-0.689
PESQ-NB : 3.409+/-0.481
SNR : -5.140+/-0.379
SSNR : -6.903+/-0.252
STOI : 0.569+/-0.051
OPed to txt
Models saved
Saving model....


 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [09:11<03:02, 182.99s/it]


Sample metrics computed
Averages computed
Metrics on test data (cleaned by model)
PESQ-WB : 3.182+/-0.761
PESQ-NB : 3.760+/-0.391
SNR : -5.352+/-0.252
SSNR : -7.095+/-0.170
STOI : 0.597+/-0.039
OPed to txt
Models saved
Saving model....


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [12:11<00:00, 182.98s/it]


Sample metrics computed
Averages computed
Metrics on test data (cleaned by model)
PESQ-WB : 3.156+/-0.768
PESQ-NB : 3.759+/-0.393
SNR : -5.345+/-0.259
SSNR : -7.086+/-0.175
STOI : 0.597+/-0.039
OPed to txt
Models saved


In [35]:
model_weights_path = "marine_Noise2Noise/Weights/dc20_model_4.pth"

dcunet20 = DCUnet20(N_FFT, HOP_LENGTH).to(DEVICE)
optimizer = torch.optim.Adam(dcunet20.parameters())

checkpoint = torch.load(model_weights_path,
                        map_location=torch.device('cpu')
                       )

## Inferencia

In [36]:
from pydub import AudioSegment
index = 0
for a in os.listdir('marine_Noise2Noise/Samples/'):
    if len(a.split('.'))==2:
        print(a.split('.wav')[0])
        os.mkdir('marine_Noise2Noise/Samples/'+str(a.split('.wav')[0]))
    sound = AudioSegment.from_wav("marine_Noise2Noise/Samples/"+a)
    audio=[]
    for b in range(0,len(sound),3400):
        extract = sound[b:b+3400]
        extract.export('marine_Noise2Noise/Samples/'+str(a.split('.wav')[0])+str('/'+str(b)+a.split('.wav')[0])+'.wav', format="wav")
        file=str(b)+a
        test_noisy_files = sorted(list(Path(f"marine_Noise2Noise/Samples/{a.split('.wav')[0]}/").rglob(file)))
        test_dataset = SpeechDataset(test_noisy_files, test_clean_files, N_FFT, HOP_LENGTH)
        test_loader_single_unshuffled = DataLoader(test_dataset, batch_size=1, shuffle=False)
        dcunet20.eval()
        test_loader_single_unshuffled_iter = iter(test_loader_single_unshuffled)
        x_n, x_c = next(test_loader_single_unshuffled_iter)
        for _ in range(index):
            x_n, x_c = next(test_loader_single_unshuffled_iter)
        x_est = dcunet20(x_n.to(DEVICE), is_istft=True)
        x_est_np = x_est[0].view(-1).detach().cpu().numpy()
        audio.append(x_est_np)
    path='marine_Noise2Noise/Samples/'+'/denoise'+str(a.split('.wav')[0])+'.wav'
    noise_addition_utils.save_audio_file(np_array=audio,file_path=Path(path), sample_rate=SAMPLE_RATE, bit_precision=16)

06617025fc3b16ee10bae2755634fc81
1947ce80f94154d6085084a574de5474
231c5f04e68be49c0a052bf222f13ae9
2df0aaeb652216469c343b814ec429cd
3509010242589e4bf2e54239e08ff322
42401006318d93fdd03eb88a67930a72
5df9b4772b0b501e0fb2b3ec7f749771
76302d5f26e67d091a6d6cb120fe78ae
79c83705430206469aa5d88a4581c968
8246a7991f4eead7094882dba0871b17
9fb046485fa8745f908db86e2e89f8c6
a2b2c78f2de1b6c29c5cac33b72366e3
ab8b9a2196d6d96831fbf8cbd2d13a6c
b1315e779778885a511668ceb349d32b
b2cc12c06ee92cd1ee1ed452087f6136
db581f1fe0ae54b48e0033d1f8bc509f
e3c63621e843cf16dfff8f42bd6c9015
e88555d41934d6c782cdf542bfbbf58b
e91c8b374bcd86190632dd2206b7aedb
f9175beb21dd1394a387af49ab505bcb


# Aplicacion de algoritmo clasico de reduccion de ruido

In [37]:
os.listdir('marine_Noise2Noise/Samples')

['denoise06617025fc3b16ee10bae2755634fc81.wav',
 'denoise1947ce80f94154d6085084a574de5474.wav',
 'denoise231c5f04e68be49c0a052bf222f13ae9.wav',
 'denoise2df0aaeb652216469c343b814ec429cd.wav',
 'denoise3509010242589e4bf2e54239e08ff322.wav',
 'denoise42401006318d93fdd03eb88a67930a72.wav',
 'denoise5df9b4772b0b501e0fb2b3ec7f749771.wav',
 'denoise76302d5f26e67d091a6d6cb120fe78ae.wav',
 'denoise79c83705430206469aa5d88a4581c968.wav',
 'denoise8246a7991f4eead7094882dba0871b17.wav',
 'denoise9fb046485fa8745f908db86e2e89f8c6.wav',
 'denoisea2b2c78f2de1b6c29c5cac33b72366e3.wav',
 'denoiseab8b9a2196d6d96831fbf8cbd2d13a6c.wav',
 'denoiseb1315e779778885a511668ceb349d32b.wav',
 'denoiseb2cc12c06ee92cd1ee1ed452087f6136.wav',
 'denoisedb581f1fe0ae54b48e0033d1f8bc509f.wav',
 'denoisee3c63621e843cf16dfff8f42bd6c9015.wav',
 'denoisee88555d41934d6c782cdf542bfbbf58b.wav',
 'denoisee91c8b374bcd86190632dd2206b7aedb.wav',
 'denoisef9175beb21dd1394a387af49ab505bcb.wav',
 'e3c63621e843cf16dfff8f42bd6c9015.wav',

In [46]:
from scipy.io import wavfile
import noisereduce as nr

In [47]:
for a in os.listdir('marine_Noise2Noise/Samples'):
    if len(a.split('.'))==2:
        print('marine_Noise2Noise/Samples/{}'.format(a))
        sr,data = wavfile.read('marine_Noise2Noise/Samples/{}'.format(a))
        data=data*4
        reduced_noise = nr.reduce_noise(y=data, sr=sr,prop_decrease=0.7)
        wavfile.write('marine_Noise2Noise/Samples/NONOISE_{}'.format(a), sr, reduced_noise)

marine_Noise2Noise/Samples/denoise06617025fc3b16ee10bae2755634fc81.wav
marine_Noise2Noise/Samples/denoise1947ce80f94154d6085084a574de5474.wav
marine_Noise2Noise/Samples/denoise231c5f04e68be49c0a052bf222f13ae9.wav
marine_Noise2Noise/Samples/denoise2df0aaeb652216469c343b814ec429cd.wav
marine_Noise2Noise/Samples/denoise3509010242589e4bf2e54239e08ff322.wav
marine_Noise2Noise/Samples/denoise42401006318d93fdd03eb88a67930a72.wav
marine_Noise2Noise/Samples/denoise5df9b4772b0b501e0fb2b3ec7f749771.wav
marine_Noise2Noise/Samples/denoise76302d5f26e67d091a6d6cb120fe78ae.wav
marine_Noise2Noise/Samples/denoise79c83705430206469aa5d88a4581c968.wav
marine_Noise2Noise/Samples/denoise8246a7991f4eead7094882dba0871b17.wav
marine_Noise2Noise/Samples/denoise9fb046485fa8745f908db86e2e89f8c6.wav
marine_Noise2Noise/Samples/denoisea2b2c78f2de1b6c29c5cac33b72366e3.wav
marine_Noise2Noise/Samples/denoiseab8b9a2196d6d96831fbf8cbd2d13a6c.wav
marine_Noise2Noise/Samples/denoiseb1315e779778885a511668ceb349d32b.wav
marine

Los archivos NONOISE_DENOISE son aquellos pasados tanto por la red neuronal como por el procesimiento clasico, los DENOISE solo por la red neuronal, los NOISE solo por los metodos clasicos